This notebook downloads the data and saves it as a df and a db in postgresql

In [1]:
import urllib.request,io
import requests
import urllib
import json
import sys
import zipfile
import shutil
import timeit
import os
import pickle

In [3]:
import pandas as pd
import numpy as np

In [3]:
# Make dirs
if not os.path.exists('data'):
    os.makedirs('data')
    
if not os.path.exists('progress'):
    os.makedirs('progress')
    
if not os.path.exists('progress/wrangled'):
    os.makedirs('progress/wrangled')

if not os.path.exists('progress/modeling'):
    os.makedirs('progress/modeling')
    
if not os.path.exists('progress/wrangled/percent_1'):
    os.makedirs('progress/wrangled/percent_1')

## 1. Downloading the Data

In [4]:
# Archive of downloadable links from FDA
# Data are available for download at this site: https://open.fda.gov/tools/downloads/

# For a cursory look at the data, see this link
# https://open.fda.gov/apis/drug/event/explore-the-api-with-an-interactive-chart/

# For information on entries see this link
# https://open.fda.gov/apis/drug/event/searchable-fields

jsonurl = 'https://api.fda.gov/download.json'
with urllib.request.urlopen(jsonurl) as url:
    data = json.loads(url.read().decode())

In [5]:
# Disclaimer
data['meta']['disclaimer']

'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.'

In [6]:
# Entries of downloadable files
# 760 in total

# Index key:
# 2008 Q2: 97:104

# 2017 Q1: 613:640
# 2017 Q2: 640:666
# 2017 Q3: 666:693
# 2017 Q4: 693:720

partitions = data['results']['drug']['event']['partitions']
# Display a few partitions
partitions[613:720]

[{'size_mb': '3.19',
  'records': 12000,
  'display_name': '2017 Q1 (part 1 of 27)',
  'file': 'https://download.open.fda.gov/drug/event/2017q1/drug-event-0001-of-0027.json.zip'},
 {'size_mb': '3.19',
  'records': 12000,
  'display_name': '2017 Q1 (part 2 of 27)',
  'file': 'https://download.open.fda.gov/drug/event/2017q1/drug-event-0002-of-0027.json.zip'},
 {'size_mb': '1.90',
  'records': 12000,
  'display_name': '2017 Q1 (part 3 of 27)',
  'file': 'https://download.open.fda.gov/drug/event/2017q1/drug-event-0003-of-0027.json.zip'},
 {'size_mb': '2.48',
  'records': 12000,
  'display_name': '2017 Q1 (part 4 of 27)',
  'file': 'https://download.open.fda.gov/drug/event/2017q1/drug-event-0004-of-0027.json.zip'},
 {'size_mb': '3.14',
  'records': 12000,
  'display_name': '2017 Q1 (part 5 of 27)',
  'file': 'https://download.open.fda.gov/drug/event/2017q1/drug-event-0005-of-0027.json.zip'},
 {'size_mb': '3.70',
  'records': 12000,
  'display_name': '2017 Q1 (part 6 of 27)',
  'file': 'http

In [7]:
total_records = 0
total_size = 0
urls = []
partitions_slice = partitions
for entry in partitions_slice:
    urls.append(entry['file'])
    total_records = total_records + entry['records']
    total_size = total_size + float(entry['size_mb'])
print('{records:0.2f} million records for a total size of {size:0.2f} GB'.format(records = total_records/10**6, size = total_size/1000))

9.14 million records for a total size of 42.85 GB


In [8]:
# Save compressed files in data folder
file_name = []
j = 0
#for i in range(0,len(urls)): # This goes through all the data
print('Progress...')
for i in range(97,104):
    start_time = timeit.default_timer()
    print('File {num} with size = {size} MB downloading...'.format(num = i, size = partitions[i]['size_mb']))
    url = urls[i] # Get the url
    # Define the filename for the data folder
    file_name.append('./data/' + url.split('/')[-2] + '_' + url.split('/')[-1]) 
    # Download the data and save it to the data folder
    with urllib.request.urlopen(url) as response, open(file_name[j], 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
    j += 1
    elapsed = timeit.default_timer() - start_time
    print('Completed in {time:0.2f} seconds'.format(time = elapsed))
print('Done.')

Progress...
File 97 with size = 2.00 MB downloading...
Completed in 0.73 seconds
File 98 with size = 3.38 MB downloading...
Completed in 0.63 seconds
File 99 with size = 6.28 MB downloading...
Completed in 0.89 seconds
File 100 with size = 44.29 MB downloading...
Completed in 2.84 seconds
File 101 with size = 125.08 MB downloading...
Completed in 7.09 seconds
File 102 with size = 119.46 MB downloading...
Completed in 6.56 seconds
File 103 with size = 22.91 MB downloading...
Completed in 1.60 seconds
Done.


In [9]:
# Unzip the data
data_2 = []
data_inter = None  
j = 0
# Run through sample
print('Progress...')
for j in range(0,len(file_name)):
    start_time = timeit.default_timer()
    with zipfile.ZipFile((file_name[j]), "r") as z: # get the .zip file
        for filename in z.namelist():  
            with z.open(filename) as f:  # Open the file
                print('File {iterable} of {total} loading...'.format(iterable = j+1, total = len(file_name)))  # print a progress check
                data_inter = f.read()   # read the file
                data_2.append(json.loads(data_inter.decode("utf-8"))) # decode and save
                elapsed = timeit.default_timer() - start_time
                print('{num_records} records loaded in {time:0.2f} seconds.'.format(num_records = len(data_2[j]['results']), time= elapsed))
print('Done.')        

Progress...
File 1 of 7 loading...
12000 records loaded in 0.47 seconds.
File 2 of 7 loading...
12000 records loaded in 0.79 seconds.
File 3 of 7 loading...
12000 records loaded in 0.90 seconds.
File 4 of 7 loading...
12000 records loaded in 2.61 seconds.
File 5 of 7 loading...
12000 records loaded in 5.83 seconds.
File 6 of 7 loading...
12000 records loaded in 5.41 seconds.
File 7 of 7 loading...
2155 records loaded in 0.88 seconds.
Done.


In [10]:
# File information
type_file = type(data_2)
type_entry = type(data_2[0]['results'][0])

size = 0
for i in range(0, len(data_2)):
    for j in range(0, len(data_2[i]['results'])):
        size = size + sys.getsizeof(data_2[i]['results'][j])
print('data_2 is a {} where entries of data_2[0][\'results\'][0] are {}'.format(type_file, type_entry))
print('Approximate Size of packed file is {size:.2f} MB'.format(size = size/10**6))

data_2 is a <class 'list'> where entries of data_2[0]['results'][0] are <class 'dict'>
Approximate Size of packed file is 48.05 MB


**Save**

In [11]:
# General, filenames used in a general set of records
#with open("./progress/file_name.txt", "wb") as fp:   #Pickling
#    pickle.dump(file_name, fp)

In [12]:
# General 
#with open("./progress/data_2.txt", "wb") as fp:   #Pickling
#    pickle.dump(data_2, fp)

In [13]:
#with open("./progress/file_name_sampleforunique.txt", "wb") as fp:   #Pickling
#    pickle.dump(file_name, fp)

In [14]:
#with open("./progress/data_sampleforunique.txt", "wb") as fp:   #Pickling
#    pickle.dump(data_2, fp)

**Load**

In [15]:
# General
#with open("./progress/file_name.txt", "rb") as fp:   # Unpickling
#    file_name = pickle.load(fp)

In [16]:
# General
#with open("./progress/data_2.txt", "rb") as fp:   # Unpickling
#    data_2 = pickle.load(fp)

In [17]:
# 2008
#with open("./progress/file_name_2008.txt", "rb") as fp:   # Unpickling
#    file_name = pickle.load(fp)

In [18]:
# 2008
#with open("./progress/data_2008.txt", "rb") as fp:   # Unpickling
#    data_2 = pickle.load(fp)

**File Information**

In [19]:
type_file = type(data_2)
type_entry = type(data_2[0]['results'][0])
txt_size = os.path.getsize('./progress/data_2.txt')

size = 0
for i in range(0, len(data_2)):
    for j in range(0, len(data_2[i]['results'])):
        size = size + sys.getsizeof(data_2[i]['results'][j])
print('Number of files = {}'.format(len(file_name)))
print('data_2 is a {} where entries of data_2[0][\'results\'][0] are {}'.format(type_file, type_entry))
print('Approximate Size of packed file is {size:.2f} MB'.format(size = size/10**6))
print('Total Size of stored txt file is {size:.2f} MB'.format(size = txt_size/10**6))

Number of files = 7
data_2 is a <class 'list'> where entries of data_2[0]['results'][0] are <class 'dict'>
Approximate Size of packed file is 48.05 MB
Total Size of stored txt file is 1054.11 MB


## 2. Convert to a DataFrame

In [20]:
from helper_funcs.munging_func import section_1, section_2, section_3, section_4, section_5, section_6, section_7

In [21]:
df_1 = section_1(data_2, file_name)
df_2 = section_2(data_2)
df_3 = section_3(data_2)
df_4 = section_4(data_2)
df_5 = section_5(data_2)
df_6 = section_6(data_2)
df_7 = section_7(data_2)

Section 1, file 1 of 7...
12.73 ms to complete
Section 1, file 2 of 7...
12.67 ms to complete
Section 1, file 3 of 7...
12.35 ms to complete
Section 1, file 4 of 7...
12.99 ms to complete
Section 1, file 5 of 7...
13.66 ms to complete
Section 1, file 6 of 7...
13.19 ms to complete
Section 1, file 7 of 7...
2.41 ms to complete
Converting lists to dataframe...
3018.30 ms to complete.

Section 2, file 1 of 7...
95.88 ms to complete
Section 2, file 2 of 7...
115.51 ms to complete
Section 2, file 3 of 7...
126.47 ms to complete
Section 2, file 4 of 7...
225.27 ms to complete
Section 2, file 5 of 7...
325.69 ms to complete
Section 2, file 6 of 7...
327.72 ms to complete
Section 2, file 7 of 7...
60.05 ms to complete
Converting lists to dataframe...
4676.32 ms to complete.

Section 3, file 1 of 7...
109.74 ms to complete
Section 3, file 2 of 7...
244.50 ms to complete
Section 3, file 3 of 7...
354.51 ms to complete
Section 3, file 4 of 7...
2683.41 ms to complete
Section 3, file 5 of 7...
814

In [22]:
df = df_1.join(df_2).join(df_3).join(df_4).join(df_5).join(df_6).join(df_7)

In [23]:
df.shape

(74155, 88)

## 3. Customize DataFrame

In [24]:
# Drop unnecessary columns
df = df[['patient_onset_age', 'patient_weight', 'patient_sex', 'generic_name', 'drug_char', 
         'drug_indication', 'admin_route', 'reaction_medDRA_pt', 'serious', 'seriousness_congential_anomali', 
         'seriousness_death', 'seriousness_disabling', 'seriousness_hospitalization', 
         'seriousness_lifethreatening', 'seriousness_other']]

In [25]:
df.head()

,patient_onset_age,patient_weight,patient_sex,generic_name,drug_char,drug_indication,admin_route,reaction_medDRA_pt,serious,seriousness_congential_anomali,seriousness_death,seriousness_disabling,seriousness_hospitalization,seriousness_lifethreatening,seriousness_other
0,NA,NA,2,_.._TAMSULOSIN HYDROCHLORIDE,_1,NA,NA,_._SYNCOPE,2,0,0,0,0,0,0
1,81,68.4932,1,_.__.._TAMSULOSIN HYDROCHLORIDE_._,_1_.__1_._,_GOUT_.__BENIGN PROSTATIC HYPERPLASIA_._,_048_.__048_._,_._DRUG INEFFECTIVE_._PRURITUS_._RASH GENERALI...,1,0,0,0,0,0,1
2,14,NA,2,_.._INSULIN GLARGINE_.__._,_1_.__2_._,_.__DIABETES MELLITUS_._,_058_.__._,_._WEIGHT INCREASED,1,0,0,0,0,0,1
3,NA,14.9687,2,_.._CETIRIZINE HYDROCHLORIDE,_1,_SEASONAL ALLERGY,NA,_._ABNORMAL BEHAVIOUR_._AGGRESSION_._CATATONIA...,2,0,0,0,0,0,0
4,58,80,2,_.._MINOCYCLINE HYDROCHLORIDE,_1,_GINGIVAL DISORDER,NA,_._CHEST PAIN_._DRUG HYPERSENSITIVITY_._DRY MO...,1,0,0,1,0,1,0


In [26]:
# Check percent of missing values
df.replace('NA', np.nan).isna().sum()/len(df)

patient_onset_age                 0.520531
patient_weight                    0.676785
patient_sex                       0.078727
generic_name                      0.062005
drug_char                         0.000000
drug_indication                   0.138184
admin_route                       0.170157
reaction_medDRA_pt                0.000000
serious                           0.000000
seriousness_congential_anomali    0.000000
seriousness_death                 0.000000
seriousness_disabling             0.000000
seriousness_hospitalization       0.000000
seriousness_lifethreatening       0.000000
seriousness_other                 0.000000
dtype: float64

**NA comments**

- Lots of NA in the patient characteristics (Age, Weight, sex). Will likely have to run separate models for each one.
- If generic_name is gone, the row has to go. No use with it. 
- If drug_indication is NA, we can fill the one-hot-encoders with zeros. Just no reason to take the drug!

In [27]:
# Drop missing generic names
df['generic_name'] = df['generic_name'].replace('NA', np.nan)
df = df.dropna(axis=0).reset_index()
df.shape

(69557, 16)

## 4. Feature Engineering

In [28]:
from helper_funcs.wrangling_func import unique_gen, entries_col_to_lists, value_gen, data_wrangling, percent_gen
from helper_funcs.wrangling_func import one_hot_encode_drugs, entry_condenser
from helper_funcs.conversion_dicts import quals, patientsex, admin_route_dict, drug_char_dict, unii_dict
from helper_funcs.conversion_dicts import dict_LLT_PT, dict_PT_HLT, dict_HLT_HLGT, dict_HLGT_SOC

/home/spwhite1337/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [29]:
df_ML = pd.DataFrame()

In [30]:
# List for the length of all the entry types
entries_len = []

**generic_name**

In [31]:
# One-hot-encode with raw input (~2000 drugs)
# df_tmp = one_hot_encode_drugs('generic_name', df)
# df_tmp.head()

In [32]:
test_col = 'generic_name'

### One-hot-encode with condensed input
# Generate list of unique drugs
unique_entries = unique_gen(test_col, df)
# Condense this list to a lower dimensional space
cond_dict = entry_condenser(unique_entries, 25)
# Get raw columns as a nested list
col_as_list = entries_col_to_lists(test_col, df)
# Convert raw column to lower-d space
col_list_cond = [[[cond_dict[l1] for l1 in l2 if l1 in cond_dict.keys()] for l2 in l3] for l3 in col_as_list]
# Unnest the lowest list
col_list_cond = [[x for y in entry for x in y] for entry in col_list_cond]

# One hot encoder
# For more information, see the following link:
# https://stackoverflow.com/questions/46864816/convert-data-frame-of-comma-separated-strings-to-one-hot-encoded
print('One Hot Encoding...')
df_tmp = pd.Series(col_list_cond).str.join(',').str.split(',',expand = True).apply(pd.Series.value_counts, 1).iloc[:,:].fillna(0)
print('Done.')
df_tmp.head()

Converting series to list...
Removing commas...
Converting to list of lists
One Hot Encoding...
Done.


,,5%,ABACAVIR,ABACAVIR SULFATE,ACETAMINOPHEN AND CODEINE,ADENOSINE,ALCOHOL,ALENDRONATE SODIUM,ALISKIREN HEMIFUMARATE AN,ALUMINUM HYDROXIDE AND MA,...,TECHNETIUM TC 99M-LABELED,TETRACAINE HYDROCHLORIDE,TETRAHYDROZOLINE HYDROCHL,THALLOUS CHLORIDE TL 201,TRETINOIN,ULEX EUROPAEUS WHOLE,VITAMIN,VITAMINS AND MINERALS,WART REMOVER LIQUID,ZINC OXIDE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# clean up
df_tmp = df_tmp.drop(['5%'], axis=1)
df_tmp = df_tmp.drop([''], axis = 1)

In [34]:
# start the final df
entries_len.append(len(df_tmp.columns))
df_ML = df_tmp.copy()

In [35]:
df_ML.head()

,ABACAVIR,ABACAVIR SULFATE,ACETAMINOPHEN AND CODEINE,ADENOSINE,ALCOHOL,ALENDRONATE SODIUM,ALISKIREN HEMIFUMARATE AN,ALUMINUM HYDROXIDE AND MA,AMLODIPINE BESYLATE AND V,ANTI-THYMOCYTE GLOBULIN (,...,TECHNETIUM TC 99M-LABELED,TETRACAINE HYDROCHLORIDE,TETRAHYDROZOLINE HYDROCHL,THALLOUS CHLORIDE TL 201,TRETINOIN,ULEX EUROPAEUS WHOLE,VITAMIN,VITAMINS AND MINERALS,WART REMOVER LIQUID,ZINC OXIDE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**drug_char**

In [36]:
# one-hot-encode
df_tmp = one_hot_encode_drugs('drug_char', df)
df_tmp.head()

Converting series to list...
Removing commas...
Converting to list of lists
One Hot Encoding...
Done.


,Concominant,Interacting,Suspect,Unknown
0,0.0,0.0,1.0,0.0
1,0.0,0.0,2.0,1.0
2,1.0,0.0,1.0,1.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [37]:
# clean up
df_tmp = df_tmp.drop(['Unknown'], axis = 1)

In [38]:
# join to the rest
entries_len.append(len(df_tmp.columns))
df_ML = pd.concat([df_ML, df_tmp], axis=1, join_axes=[df_ML.index])

In [39]:
df_ML.head()

,ABACAVIR,ABACAVIR SULFATE,ACETAMINOPHEN AND CODEINE,ADENOSINE,ALCOHOL,ALENDRONATE SODIUM,ALISKIREN HEMIFUMARATE AN,ALUMINUM HYDROXIDE AND MA,AMLODIPINE BESYLATE AND V,ANTI-THYMOCYTE GLOBULIN (,...,THALLOUS CHLORIDE TL 201,TRETINOIN,ULEX EUROPAEUS WHOLE,VITAMIN,VITAMINS AND MINERALS,WART REMOVER LIQUID,ZINC OXIDE,Concominant,Interacting,Suspect
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


**drug_indication**

In [40]:
# All drug_indications (~7000)
# df_tmp = one_hot_encode_drugs('drug_indication', df)
# df_tmp.head()

In [41]:
test_col = 'drug_indication'

### One-hot-encode with condensed input
# Generate list of unique drugs
unique_entries = unique_gen(test_col, df)
# Condense this list to a lower dimensional space
cond_dict = entry_condenser(unique_entries, 25)
# Get raw columns as a nested list
col_as_list = entries_col_to_lists(test_col, df)
# Convert raw column to lower-d space
col_list_cond = [[[cond_dict[l1] for l1 in l2 if l1 in cond_dict.keys()] for l2 in l3] for l3 in col_as_list]
# Unnest the lowest list
col_list_cond = [[x for y in entry for x in y] for entry in col_list_cond]

# One hot encoder
# For more information, see the following link:
# https://stackoverflow.com/questions/46864816/convert-data-frame-of-comma-separated-strings-to-one-hot-encoded
print('One Hot Encoding...')
df_tmp = pd.Series(col_list_cond).str.join(',').str.split(',',expand = True).apply(pd.Series.value_counts, 1).iloc[:,:].fillna(0)
print('Done.')
df_tmp.head()

Converting series to list...
Removing commas...
Converting to list of lists
One Hot Encoding...
Done.


,,ABDOMINAL SEPSIS,ACUTE LYMPHOCYTIC LEUKAEM,ACUTE MYELOID LEUKAEMIA,ADDISON'S DISEASE,ADENOMYOSIS,ADRENAL CARCINOMA,ADRENAL INSUFFICIENCY,AFFECTIVE DISORDER,AGITATION,...,UTERINE MASS,VARICES OESOPHAGEAL,VASCULAR OPERATION,VENOUS THROMBOSIS,VENTRICULAR TACHYCARDIA,VITAMIN C DEFICIENCY,VITAMIN SUPPLEMENTATION,VULVOVAGINAL ADHESION,WEIGHT DECREASED,WOUND INFECTION PSEUDOMON
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# clean up
df_tmp = df_tmp.drop([''], axis = 1)

In [43]:
# join to the rest
entries_len.append(len(df_tmp.columns))
df_ML = pd.concat([df_ML, df_tmp], axis=1, join_axes=[df_ML.index])

In [44]:
df_ML.head()

,ABACAVIR,ABACAVIR SULFATE,ACETAMINOPHEN AND CODEINE,ADENOSINE,ALCOHOL,ALENDRONATE SODIUM,ALISKIREN HEMIFUMARATE AN,ALUMINUM HYDROXIDE AND MA,AMLODIPINE BESYLATE AND V,ANTI-THYMOCYTE GLOBULIN (,...,UTERINE MASS,VARICES OESOPHAGEAL,VASCULAR OPERATION,VENOUS THROMBOSIS,VENTRICULAR TACHYCARDIA,VITAMIN C DEFICIENCY,VITAMIN SUPPLEMENTATION,VULVOVAGINAL ADHESION,WEIGHT DECREASED,WOUND INFECTION PSEUDOMON
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**admin_route**

In [45]:
# one-hot-encode
df_tmp = one_hot_encode_drugs('admin_route', df)
df_tmp.head()

Converting series to list...
Removing commas...
Converting to list of lists
One Hot Encoding...
Done.


,Auricular,Buccal,Cutaneous,Dental,Endotracheal,Epidural,Hemodialysis,Intra corpus cavernosum,Intra-amniotic,Intra-arterial,...,Subdermal,Sublingual,Sunconjunctival,Topical,Transdermal,Transmammary,Transplacental,Unknown,Urethral,Vaginal
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# Clean up
df_tmp = df_tmp.drop(['Unknown'], axis = 1)
df_tmo = df_tmp.drop(['Not Listed'], axis = 1)

In [47]:
# join to the rest
entries_len.append(len(df_tmp.columns))
df_ML = pd.concat([df_ML, df_tmp], axis=1, join_axes=[df_ML.index])

In [48]:
df_ML.head()

,ABACAVIR,ABACAVIR SULFATE,ACETAMINOPHEN AND CODEINE,ADENOSINE,ALCOHOL,ALENDRONATE SODIUM,ALISKIREN HEMIFUMARATE AN,ALUMINUM HYDROXIDE AND MA,AMLODIPINE BESYLATE AND V,ANTI-THYMOCYTE GLOBULIN (,...,Subcutaneous,Subdermal,Sublingual,Sunconjunctival,Topical,Transdermal,Transmammary,Transplacental,Urethral,Vaginal
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**reaction_medDRA_pt**

In [49]:
# Convert df column to a list
col_list = entries_col_to_lists('reaction_medDRA_pt', df)
# To HLT
col_list_HLT = [[[dict_PT_HLT[l1] for l1 in l2 if l1 in dict_PT_HLT.keys()] for l2 in l3] for l3 in col_list]
# To HLGT
col_list_HLGT = [[[dict_HLT_HLGT[l1] for l1 in l2 if l1 in dict_HLT_HLGT.keys()] for l2 in l3] for l3 in col_list_HLT]
# To SOC
col_list_SOC = [[[dict_HLGT_SOC[l1] for l1 in l2 if l1 in dict_HLGT_SOC.keys()] for l2 in l3] for l3 in col_list_HLGT]

Converting series to list...
Removing commas...
Converting to list of lists


In [50]:
# Undo the first nested layer

# For SOC
#col_list_ML = [[x for y in entry for x in y] for entry in col_list_SOC]
# For HLGT
#col_list_ML = [[x for y in entry for x in y] for entry in col_list_HLGT]
# For HLT
col_list_ML = [[x for y in entry for x in y] for entry in col_list_HLT]
# For PT
# col_list_ML = [[x for y in entry for x in y] for entry in col_list]

# One hot encoder
# For more information, see the following link:
# https://stackoverflow.com/questions/46864816/convert-data-frame-of-comma-separated-strings-to-one-hot-encoded
print('One Hot Encoding...')
df_tmp = pd.Series(col_list_ML).str.join(',').str.split(',',expand = True).apply(pd.Series.value_counts, 1).iloc[:,:].fillna(0)
print('Done.')
df_tmp.head()

One Hot Encoding...
Done.


,,BACTERAEMIA,BLOATING AND DISTENSION,CHILDHOOD AND ADOLESCENCE PSYCHIATRIC DISORDERS NEC,CHOROID AND VITREOUS INFECTIONS AND INFLAMMATIONS,DELIVERY AND LACTATION,DEPOSIT AND DEGENERATION,DEPOSIT AND DEGENERATION (EXCL CATARACTS),DEPOSIT AND DEGENERATION OF EYE NEC,DRUGS AND OTHER CHEMICALS,...,VULVOVAGINAL CYSTS AND NEOPLASMS,VULVOVAGINAL DISORDERS NEC,VULVOVAGINAL SIGNS AND SYMPTOMS,WALDENSTROM'S MACROGLOBULINAEMIAS,WATER AND ELECTROLYTE ANALYSES NEC,WATER SOLUBLE VITAMIN DEFICIENCIES,WHITE BLOOD CELL ABNORMAL FINDINGS NEC,WHITE BLOOD CELL ANALYSES,WITHDRAWAL AND REBOUND EFFECTS,YERSINIA INFECTIONS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# clean up
df_tmp = df_tmp.drop([''], axis = 1)

In [52]:
# joing to the rest
entries_len.append(len(df_tmp.columns))
df_ML = pd.concat([df_ML, df_tmp], axis=1, join_axes=[df_ML.index])

In [53]:
df_ML.head()

,ABACAVIR,ABACAVIR SULFATE,ACETAMINOPHEN AND CODEINE,ADENOSINE,ALCOHOL,ALENDRONATE SODIUM,ALISKIREN HEMIFUMARATE AN,ALUMINUM HYDROXIDE AND MA,AMLODIPINE BESYLATE AND V,ANTI-THYMOCYTE GLOBULIN (,...,VULVOVAGINAL CYSTS AND NEOPLASMS,VULVOVAGINAL DISORDERS NEC,VULVOVAGINAL SIGNS AND SYMPTOMS,WALDENSTROM'S MACROGLOBULINAEMIAS,WATER AND ELECTROLYTE ANALYSES NEC,WATER SOLUBLE VITAMIN DEFICIENCIES,WHITE BLOOD CELL ABNORMAL FINDINGS NEC,WHITE BLOOD CELL ANALYSES,WITHDRAWAL AND REBOUND EFFECTS,YERSINIA INFECTIONS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 5. Save the dfs for Modeling and SQL

In [54]:
df_ML.shape

(69557, 1786)

In [55]:
# Add in the outcome variables
df_outcomes = df[['serious', 'seriousness_congential_anomali', 
         'seriousness_death', 'seriousness_disabling', 'seriousness_hospitalization', 
         'seriousness_lifethreatening', 'seriousness_other']]

df_ML = pd.concat([df_ML, df_outcomes], axis = 1, join_axes = [df_ML.index])

In [ ]:
# Convert all columns to numeric
df_ML = df_ML.apply(pd.to_numeric)
df_ML['serious'] = df_ML['serious'] - 1

In [ ]:
# Visualize the sparsity of the matrix
import matplotlib.pyplot as plt
fig, axs = plt.subplots(figsize = (8,8))
axs.spy(df_ML.head(500), markersize=1)
plt.show()

In [ ]:
# Save as a csv
#df_ML.to_csv('./progress/modeling/df_ML_model_1_1.csv')

In [ ]:
# save the length of entries
with open("./progress/modeling/entries_len_1_3.txt", "wb") as fp:   #Pickling
    pickle.dump(entries_len, fp)

In [ ]:
entries_len

**TO SQL**

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [ ]:
df_ML = pd.read_csv('./progress/modeling/df_ML_sample.csv', index_col = 0)
df_ML.head()

In [ ]:
# Remove special characters in columns for SQL compatibility
# ||, -, *, /, <>, <, >, ,(comma), =, <=, >=, ~=, !=, ^=, (, )
df_ML.columns = df_ML.columns.str.replace('(','')
df_ML.columns = df_ML.columns.str.replace(')','')
df_ML.columns = df_ML.columns.str.replace('\'','')
df_ML.columns = df_ML.columns.str.replace('^','')
df_ML.columns = df_ML.columns.str.replace('-','')
df_ML.columns = df_ML.columns.str.replace(' ','_')

In [ ]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username/password, and connection specifics
username = 'postgres'
password = 'indy'     # change this
host     = 'localhost'
port     = '5432'            # default port that postgres listens on
db_name  = 'FDA_ML'

In [ ]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine( 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, db_name) )
print(engine.url)

In [ ]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

In [ ]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
df_ML.to_sql('FDA_ML_sampled', engine, if_exists='replace', index_label = 'Index')